In [ ]:
import numpy as np
import tensorflow as tf
import keras
import cv2
from keras.layers import MaxPool2D,Conv2D,UpSampling2D,Input,Dropout
from keras.models import Sequential
from keras.preprocessing.image import img_to_array
import os
from tqdm import tqdm
import re
import matplotlib.pyplot as plt

In [ ]:
# to get the files in proper order
def sorted_alphanumeric(data):
    convert = lambda text: int(text) if text.isdigit() else text.lower()
    alphanum_key = lambda key: [convert(c) for c in re.split('([0-9]+)',key)]
    return sorted(data,key = alphanum_key)
# defining the size of the image
SIZE = 256
color_img = []
path = '/content/drive/MyDrive/SAR/gan/v_2/agri/s2'
files = os.listdir(path)
files = sorted_alphanumeric(files)
for i in tqdm(files):
  img = cv2.imread(path + '/'+i,1)
  # open cv reads images in BGR format so we have to convert it to RGB
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  #resizing image
  img = cv2.resize(img, (SIZE, SIZE))
  img = img.astype('float32') / 255.0
  color_img.append(img_to_array(img))


gray_img = []
path = '/content/drive/MyDrive/SAR/gan/v_2/agri/s1'
files = os.listdir(path)
files = sorted_alphanumeric(files)
for i in tqdm(files):
  img = cv2.imread(path + '/'+i,1)

  #resizing image
  img = cv2.resize(img, (SIZE, SIZE))
  img = img.astype('float32') / 255.0
  gray_img.append(img_to_array(img))

100%|██████████| 4000/4000 [01:20<00:00, 49.50it/s] 


In [ ]:
train_gray_image = gray_img[:3500]
train_color_image = color_img[:3500]

test_gray_image = gray_img[3500:]
test_color_image = color_img[3500:]
# reshaping
train_g = np.reshape(train_gray_image,(len(train_gray_image),SIZE,SIZE,3))
train_c = np.reshape(train_color_image, (len(train_color_image),SIZE,SIZE,3))
print('Train color image shape:',train_c.shape)


test_gray_image = np.reshape(test_gray_image,(len(test_gray_image),SIZE,SIZE,3))
test_color_image = np.reshape(test_color_image, (len(test_color_image),SIZE,SIZE,3))
print('Test color image shape',test_color_image.shape)

Train color image shape: (3500, 160, 160, 3)
Test color image shape (500, 160, 160, 3)


In [ ]:
from keras import layers
def down(filters , kernel_size, apply_batch_normalization = True):
    downsample = tf.keras.models.Sequential()
    downsample.add(layers.Conv2D(filters,kernel_size,padding = 'same', strides = 2))
    if apply_batch_normalization:
        downsample.add(layers.BatchNormalization())
    downsample.add(keras.layers.LeakyReLU())
    return downsample


def up(filters, kernel_size, dropout = False):
    upsample = tf.keras.models.Sequential()
    upsample.add(layers.Conv2DTranspose(filters, kernel_size,padding = 'same', strides = 2))
    if dropout:
        upsample.add(layers.Dropout(0.2))
    upsample.add(keras.layers.ReLU())
    return upsample

In [ ]:
def model():
    inputs = layers.Input(shape= [SIZE,SIZE,1])
    d1 = down(128,(3,3),False)(inputs)
    d2 = down(128,(3,3),False)(d1)
    d3 = down(256,(3,3),True)(d2)
    d4 = down(512,(3,3),True)(d3)

    d5 = down(512,(3,3),True)(d4)
    #upsampling
    u1 = up(512,(3,3),False)(d5)
    u1 = layers.concatenate([u1,d4])
    u2 = up(256,(3,3),False)(u1)
    u2 = layers.concatenate([u2,d3])
    u3 = up(128,(3,3),False)(u2)
    u3 = layers.concatenate([u3,d2])
    u4 = up(128,(3,3),False)(u3)
    u4 = layers.concatenate([u4,d1])
    u5 = up(3,(3,3),False)(u4)
    u5 = layers.concatenate([u5,inputs])
    output = layers.Conv2D(3,(2,2),strides = 1, padding = 'same')(u5)
    return tf.keras.Model(inputs=inputs, outputs=output)

In [ ]:
model = model()
model.summary()

Model: "functional_10"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 160, 160, 3)    │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ sequential (Sequential)   │ (None, 80, 80, 128)    │          3,584 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ sequential_1 (Sequential) │ (None, 40, 40, 128)    │        147,584 │ sequential[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ sequential_2 (Sequential) │ (None, 20, 20, 256)    │        296,192 │ sequential_1[0][0]     │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ sequential_3 (Sequential) │ (None, 10, 10, 512)    │      1,182,208 │ sequential_2[0][0]     │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ sequential_4 (Sequential) │ (None, 5, 5, 512)      │      2,361,856 │ sequential_3[0][0]     │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ sequential_5 (Sequential) │ (None, 10, 10, 512)    │      2,359,808 │ sequential_4[0][0]     │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate (Concatenate) │ (None, 10, 10, 1024)   │              0 │ sequential_5[0][0],    │
│                           │                        │                │ sequential_3[0][0]     │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ sequential_6 (Sequential) │ (None, 20, 20, 256)    │      2,359,552 │ concatenate[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate_1             │ (None, 20, 20, 512)    │              0 │ sequential_6[0][0],    │
│ (Concatenate)             │                        │                │ sequential_2[0][0]     │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ sequential_7 (Sequential) │ (None, 40, 40, 128)    │        589,952 │ concatenate_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate_2             │ (None, 40, 40, 256)    │              0 │ sequential_7[0][0],    │
│ (Concatenate)             │                        │                │ sequential_1[0][0]     │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ sequential_8 (Sequential) │ (None, 80, 80, 128)    │        295,040 │ concatenate_2[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate_3             │ (None, 80, 80, 256)    │              0 │ sequential_8[0][0],    │
│ (Concatenate)             │                        │                │ sequential[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ sequential_9 (Sequential) │ (None, 160, 160, 3)    │          6,915 │ concatenate_3[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate_4             │ (None, 160, 160, 6)    │              0 │ sequential_9[0][0],    │
│ (Concatenate)             │                        │                │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_5 (Conv2D)         │ (None, 160, 160, 3)    │             75 │ concatenate_4[0][0]    │
└──────────────────────

 Total params: 9,602,766 (36.63 MB)

 Trainable params: 9,600,206 (36.62 MB)

 Non-trainable params: 2,560 (10.00 KB)

In [ ]:
model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = 0.001), loss = 'mean_absolute_error',
              metrics = ['acc'])

model.fit(train_g, train_c, epochs = 50, batch_size = 50)

Epoch 1/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 35s 203ms/step - acc: 0.2929 - loss: 0.2532
Epoch 2/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 25s 206ms/step - acc: 0.3951 - loss: 0.1563
Epoch 3/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 15s 210ms/step - acc: 0.4197 - loss: 0.1498
Epoch 4/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 21s 212ms/step - acc: 0.4362 - loss: 0.1453
Epoch 5/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 21s 217ms/step - acc: 0.4616 - loss: 0.1417
Epoch 6/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 21s 219ms/step - acc: 0.4806 - loss: 0.1393
Epoch 7/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 16s 223ms/step - acc: 0.4889 - loss: 0.1374
Epoch 8/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 21s 226ms/step - acc: 0.5075 - loss: 0.1357
Epoch 9/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 21s 231ms/step - acc: 0.5111 - loss: 0.1333
Epoch 10/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 16s 234ms/step - acc: 0.5478 - loss: 0.1303
Epoch 11/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 16s 231ms/step - acc: 0.5510 - loss: 0.1280
Epoch 12/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 20s 226ms/step - acc: 0.5691 - loss: 0.1247
Epoch 13/50
7

In [ ]:
model.evaluate(test_gray_image,test_color_image)

16/16 ━━━━━━━━━━━━━━━━━━━━ 9s 279ms/step - acc: 0.4791 - loss: 0.2119


[0.23403167724609375, 0.4486047625541687]

In [ ]:
# defining function to plot images pair
def plot_images(color,grayscale,predicted):
    plt.figure(figsize=(15,15))
    plt.subplot(1,3,1)
    plt.title('Color Image', color = 'green', fontsize = 20)
    plt.imshow(color)
    plt.subplot(1,3,2)
    plt.title('Grayscale Image ', color = 'black', fontsize = 20)
    plt.imshow(grayscale)
    plt.subplot(1,3,3)
    plt.title('Predicted Image ', color = 'Red', fontsize = 20)
    plt.imshow(predicted)

    plt.show()

for i in range(50,58):
    predicted = np.clip(model.predict(test_gray_image[i].reshape(1,SIZE, SIZE,3)),0.0,1.0).reshape(SIZE, SIZE,3)
    plot_images(test_color_image[i],test_gray_image[i],predicted)


In [ ]:
model.save('/content/drive/MyDrive/SAR/models/colorization-autoencoders-agri.keras')

In [ ]:
import tensorflow as tf
model = tf.keras.models.load_model('/content/drive/MyDrive/SAR/models/colorization-autoencoders-agri.keras')

In [ ]:
import numpy as np
img_g = test_gray_image[0]
img_c = test_color_image[0]
predicted = np.clip(model.predict(img.reshape(1,SIZE, SIZE,3)),0.0,1.0).reshape(SIZE, SIZE,3)

FailedPreconditionError: Graph execution error:

Detected at node StatefulPartitionedCall defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "/usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py", line 37, in <module>

  File "/usr/local/lib/python3.11/dist-packages/traitlets/config/application.py", line 992, in launch_instance

  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelapp.py", line 712, in start

  File "/usr/local/lib/python3.11/dist-packages/tornado/platform/asyncio.py", line 205, in start

  File "/usr/lib/python3.11/asyncio/base_events.py", line 608, in run_forever

  File "/usr/lib/python3.11/asyncio/base_events.py", line 1936, in _run_once

  File "/usr/lib/python3.11/asyncio/events.py", line 84, in _run

  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelbase.py", line 510, in dispatch_queue

  File "/usr/local/lib/python3.11/dist-pa

In [ ]:
import numpy as np

def calculate_psnr(y_test, y_preds):
    mse = (y_test - y_preds) ** 2
    max_pixel = 255.0
    psnr = 10 * np.log10((max_pixel ** 2) / (mse + 1e-10))
    return psnr

psnr_value = calculate_psnr(img_c, predicted)
print(f"PSNR: {np.mean(psnr_value):.2f} dB")

PSNR: 67.08 dB


In [ ]:
img_1 = tf.convert_to_tensor(img_c, dtype=tf.float32)
prediction = tf.convert_to_tensor(predicted, dtype=tf.float32)

ssim_value = tf.image.ssim(img_1, prediction, max_val=255.0)
print("SSIM (RGB vs. 3-Channel Prediction):", ssim_value.numpy())

SSIM (RGB vs. 3-Channel Prediction): 0.99445504


In [ ]:
image_for_test = test_gray_image[0]
print("Shape before squeezing:", image_for_test.shape)

Shape before squeezing: (160, 160, 3)


In [ ]:
def compute_enl(image):
    # Ensure tensor format
    image = tf.convert_to_tensor(image, dtype=tf.float32)

    # Compute mean and standard deviation
    mean = tf.reduce_mean(image)
    std = tf.math.reduce_std(image)

    # Compute ENL (avoid division by zero)
    enl = (mean ** 2) / (std ** 2 + 1e-8)  # Adding small value to avoid division by zero

    return enl.numpy()

img_1_gray = tf.image.rgb_to_grayscale(tf.convert_to_tensor(img_1, dtype=tf.float32))
prediction_gray = tf.expand_dims(prediction[..., 0], axis=-1)
image_for_test_gray = tf.image.rgb_to_grayscale(image_for_test)

enl_img_1 = compute_enl(img_1_gray)
enl_prediction = compute_enl(prediction_gray)
enl_test = compute_enl(image_for_test_gray)

print("ENL (img_1, Grayscale):", enl_img_1)
print("ENL (Prediction, First Channel):", enl_prediction)
print("ENL (Image for Test, Grayscale):", enl_test)

ENL (img_1, Grayscale): 10.362056
ENL (Prediction, First Channel): 7.8471107
ENL (Image for Test, Grayscale): 7.480365


In [ ]:
import tensorflow as tf
import numpy as np

def compute_mse(image1, image2):
    # Ensure tensor format
    image1 = tf.convert_to_tensor(image1, dtype=tf.float32)
    image2 = tf.convert_to_tensor(image2, dtype=tf.float32)

    # Compute MSE
    mse = tf.reduce_mean(tf.math.squared_difference(image1, image2))
    return mse.numpy()

mse_value = compute_mse(img_1, prediction)
print("MSE (RGB vs. 3-Channel Prediction):", mse_value)

MSE (RGB vs. 3-Channel Prediction): 0.045754164


In [ ]:
!pip install lpips

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 84.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 28.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 36.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstallin

In [ ]:
import lpips
import torch
import numpy as np

# Initialize LPIPS model (VGG-based)
lpips_model = lpips.LPIPS(net='vgg')

def compute_lpips_torch(image1, image2):
    # Convert to Tensor and normalize to [-1, 1]
    image1 = torch.tensor(image1.numpy()).permute(2, 0, 1).unsqueeze(0) / 127.5 - 1.0
    image2 = torch.tensor(image2.numpy()).permute(2, 0, 1).unsqueeze(0) / 127.5 - 1.0

    # Compute LPIPS
    lpips_distance = lpips_model(image1, image2)

    return lpips_distance.item()

# Compute LPIPS using PyTorch
lpips_value_torch = compute_lpips_torch(img_1, prediction)
print("LPIPS (PyTorch):", lpips_value_torch)


ModuleNotFoundError: No module named 'lpips'